This is the entire code used for the work flow of this project. It is also entirely available on the project website, as each chunk can be viewed under the grey "code" button.

In [126]:
#| echo: true
#| code-fold: true
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import matplotlib.pyplot as plt

API_KEY = '0da0c882151e10740c1a0a844cf845096bedb565'
c = Census(API_KEY)

acs_data = c.acs5.state_county(
    fields=[
        'NAME',         # County name
        'B01003_001E',  # Total population
        'B15003_001E',  # Population 25+
        'B19013_001E',  # Median household income
        'B19301_001E',  # Per capita income
        'B01002_001E',  # Median age
        'B02001_002E',  # White population
        'B02001_003E',  # African American population
        'B03001_003E',  # Hispanic population
        'B15003_017E',  # High school graduates
        'B15003_022E',  # Bachelor’s degree holders
        'B25077_001E',  # Median housing value
        'B17001_002E',  # Population below poverty line
    ],
    state_fips=states.PA.fips,
    county_fips="*"
)

acs_df = pd.DataFrame(acs_data)
acs_df.columns = [
    'County',  # Name of the county
    'Total Population',
    'Population Over 25',
    'Median Household Income', 
    'Per Capita Income', 
    'Median Age', 
    'White Population', 
    'African American Population', 
    'Hispanic Population', 
    'High School Graduates', 
    'Bachelors Degree Holders', 
    'Median Housing Value', 
    'Population Below Poverty Line',
    'State',  
    'County_Name'
]

acs_df['County'] = acs_df['County'].str.replace(" County, Pennsylvania", "", case=False)
acs_df['County'] = acs_df['County'].str.upper()
acs_df.drop(columns=['State'], inplace=True)

In [ ]:
#| echo: true
#| code-fold: true
geojson_path = "/Users/ryanswett/Downloads/Python/Final_Project/PaCounty2024_11.geojson"
county_data = gpd.read_file(geojson_path)
counties = county_data.merge(
    acs_df,
    left_on='FIPS_COUNT',
    right_on='County_Name',
    how='left'
)

#| echo: true
#| code-fold: true
counties['Percent_White'] = counties['White Population'] / counties['Total Population']*100 # Percent white
counties['Percent_Black'] = counties['African American Population'] / counties['Total Population']*100 # Percent black
counties['Percent_Hispanic'] = counties['Hispanic Population'] / counties['Total Population']*100 # Percent hispanic
counties['Percent_HS_degrees'] = counties['High School Graduates'] / counties['Total Population']*100 # Percent high school grads
counties['Percent_Bachelors'] = counties['Bachelors Degree Holders'] / counties['Population Over 25']*100 # Percent bachelors
counties['Percent_Poverty'] = counties['Population Below Poverty Line'] / counties['Population Over 25']*100 # Percent below poverty line

In [ ]:
#| echo: true
#| code-fold: true
parks_path = "/Users/ryanswett/Downloads/Python/Final_Project/DCNR_LocalPark202406/DCNR_LocalPark202406.shp"
parks_data = gpd.read_file(parks_path)

parks_data = parks_data.drop(columns=['STATUS', 'PARK_FEE', 'ALT_NAME', 'PREMISE_AD', 'PREMISE_CI', 'PREMISE_ZI', 'YEAR_OPEN',
                                     'PREMISE_CR', 'URL', 'COMMENTS', 'ATV', 'Basketball', 'Bicycling', 'Camping', 'Canoeing_K',
                                     'CrossCount', 'Disc_Golf', 'Dog_Park', 'Equestrian', 'Fishing', 'Fitness_Eq', 'Golf',
                                     'Hiking', 'Horseback_', 'Hunting', 'Ice_Fishin', 'Ice_Skatin', 'Motor_Boat', 'LWCF_Restr',
                                     'Mountain_B', 'Natural_Wi', 'Organized_', 'Parking', 'Pavilion', 'Pets_Allow', 'Playground',
                                     'Restrooms', 'Rock_Climb', 'Scenic_Vie', 'Sledding', 'Sports_Fie', 'Swimming', 'Tennis_Cou',
                                     'Theatre_Am', 'Trails', 'Visitor_Ce', 'White_Wate', 'Wildlife_W', 'Amenity_Co', 'Feedback_l',
                                     'Skate_Park'])

updated_parks = parks_data.groupby('PREMISE_CO')['Acres'].sum().reset_index()
updated_parks['park_sq_mi'] = updated_parks['Acres'] / 640
updated_parks['PREMISE_CO'] = updated_parks['PREMISE_CO'].str.upper()

parks = counties.merge(
    updated_parks,
    left_on='County',  
    right_on='PREMISE_CO',  
    how='left')

parks['Percent Local Park'] = parks['park_sq_mi'] / parks['AREA_SQ_MI'] * 100

In [ ]:
#| echo: true
#| code-fold: true
unemp_data = pd.read_csv("/Users/ryanswett/Downloads/Python/Final_Project/unemp.csv")
unemp_data['County'] = unemp_data['County'].str.replace('County', '', case=False).str.strip()
unemp_data['County'] = unemp_data['County'].str.upper()

pa_counties = unemp_data.merge(
    parks,
    left_on='County', 
    right_on='County', 
    how='left')

pa_counties.rename(columns={"Value (Percent)": "Unemp Rate"}, inplace=True)

pd.set_option('display.max_columns', None)
pa_counties.head()

In [ ]:
#| echo: false
# This chunk is hidden on the webiste. It just reloads the pa_counties dataframe in this notebook.
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import contextily as ctx
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D

API_KEY = '0da0c882151e10740c1a0a844cf845096bedb565'
c = Census(API_KEY)

acs_data = c.acs5.state_county(
    fields=[
        'NAME',         # County name
        'B01003_001E',  # Total population
        'B15003_001E',  # Population 25+
        'B19013_001E',  # Median household income
        'B19301_001E',  # Per capita income
        'B01002_001E',  # Median age
        'B02001_002E',  # White population
        'B02001_003E',  # African American population
        'B03001_003E',  # Hispanic population
        'B15003_017E',  # High school graduates
        'B15003_022E',  # Bachelor’s degree holders
        'B25077_001E',  # Median housing value
        'B17001_002E',  # Population below poverty line
    ],
    state_fips=states.PA.fips,
    county_fips="*"
)

acs_df = pd.DataFrame(acs_data)
acs_df.columns = [
    'County',  # Name of the county
    'Total Population',
    'Population Over 25',
    'Median Household Income', 
    'Per Capita Income', 
    'Median Age', 
    'White Population', 
    'African American Population', 
    'Hispanic Population', 
    'High School Graduates', 
    'Bachelors Degree Holders', 
    'Median Housing Value', 
    'Population Below Poverty Line',
    'State',  
    'County_Name'
]

acs_df['County'] = acs_df['County'].str.replace(" County, Pennsylvania", "", case=False)
acs_df['County'] = acs_df['County'].str.upper()
acs_df.drop(columns=['State'], inplace=True)

geojson_path = "/Users/ryanswett/Downloads/Python/Final_Project/PaCounty2024_11.geojson"
county_data = gpd.read_file(geojson_path)
counties = county_data.merge(
    acs_df,
    left_on='FIPS_COUNT',
    right_on='County_Name',
    how='left'
)

counties['Percent_White'] = counties['White Population'] / counties['Total Population']*100 # Percent white
counties['Percent_Black'] = counties['African American Population'] / counties['Total Population']*100 # Percent black
counties['Percent_Hispanic'] = counties['Hispanic Population'] / counties['Total Population']*100 # Percent hispanic
counties['Percent_HS_degrees'] = counties['High School Graduates'] / counties['Total Population']*100 # Percent high school grads
counties['Percent_Bachelors'] = counties['Bachelors Degree Holders'] / counties['Population Over 25']*100 # Percent bachelors
counties['Percent_Poverty'] = counties['Population Below Poverty Line'] / counties['Population Over 25']*100 # Percent below poverty line

parks_path = "/Users/ryanswett/Downloads/Python/Final_Project/DCNR_LocalPark202406/DCNR_LocalPark202406.shp"
parks_data = gpd.read_file(parks_path)

parks_data = parks_data.drop(columns=['STATUS', 'PARK_FEE', 'ALT_NAME', 'PREMISE_AD', 'PREMISE_CI', 'PREMISE_ZI', 'YEAR_OPEN',
                                     'PREMISE_CR', 'URL', 'COMMENTS', 'ATV', 'Basketball', 'Bicycling', 'Camping', 'Canoeing_K',
                                     'CrossCount', 'Disc_Golf', 'Dog_Park', 'Equestrian', 'Fishing', 'Fitness_Eq', 'Golf',
                                     'Hiking', 'Horseback_', 'Hunting', 'Ice_Fishin', 'Ice_Skatin', 'Motor_Boat', 'LWCF_Restr',
                                     'Mountain_B', 'Natural_Wi', 'Organized_', 'Parking', 'Pavilion', 'Pets_Allow', 'Playground',
                                     'Restrooms', 'Rock_Climb', 'Scenic_Vie', 'Sledding', 'Sports_Fie', 'Swimming', 'Tennis_Cou',
                                     'Theatre_Am', 'Trails', 'Visitor_Ce', 'White_Wate', 'Wildlife_W', 'Amenity_Co', 'Feedback_l',
                                     'Skate_Park'])

updated_parks = parks_data.groupby('PREMISE_CO')['Acres'].sum().reset_index()
updated_parks['park_sq_mi'] = updated_parks['Acres'] / 640
updated_parks['PREMISE_CO'] = updated_parks['PREMISE_CO'].str.upper()

parks = counties.merge(
    updated_parks,
    left_on='County',  
    right_on='PREMISE_CO',  
    how='left')

parks['Percent Local Park'] = parks['park_sq_mi'] / parks['AREA_SQ_MI'] * 100

unemp_data = pd.read_csv("/Users/ryanswett/Downloads/Python/Final_Project/unemp.csv")
unemp_data['County'] = unemp_data['County'].str.replace('County', '', case=False).str.strip()
unemp_data['County'] = unemp_data['County'].str.upper()

pa_counties = unemp_data.merge(
    parks,
    left_on='County', 
    right_on='County', 
    how='left')

pa_counties.rename(columns={"Value (Percent)": "Unemp Rate"}, inplace=True)

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = gpd.GeoDataFrame(pa_counties, geometry=pa_counties['geometry'])
pa_counties = pa_counties.to_crs(epsg=2272)
key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Median Household Income'].min(), vmax=pa_counties['Median Household Income'].max())
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Median Household Income',
    cmap='spring',  
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Median Income ($)"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)
plt.title("Median Household Income by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Median Housing Value'].min(), vmax=pa_counties['Median Housing Value'].max() * 1.2)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Median Housing Value',
    cmap='spring',  
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Median Housing Value ($)"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Median Housing Value by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")


plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)
key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Percent_Poverty'].min(), vmax=pa_counties['Percent_Poverty'].max())
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Percent_Poverty',
    cmap='plasma',  
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Percent Below Poverty Line"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Population Below Poverty Line by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Percent_Black'].min(), vmax=pa_counties['Percent_Black'].max() * 1.2)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Percent_Black',
    cmap='plasma',  
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Percent African American"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Percent African American by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Percent_Bachelors'].min(), vmax=pa_counties['Percent_Bachelors'].max() * 1.2)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Percent_Bachelors',
    cmap='viridis',  
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Percent Bachelors Degree"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Percent Bachelors Degrees by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Percent Local Park'].min(), vmax=pa_counties['Percent Local Park'].max() * 1.2)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Percent Local Park',
    cmap='Greens', 
    edgecolor='black',
    linewidth=0.5,
    legend=True,
    norm=norm,
    legend_kwds={'shrink': 0.6, 'label': "Percent Local Park"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='black', markersize=50, edgecolor='black', label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='red', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='white', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Percent Local Park by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties = pa_counties.to_crs(epsg=2272)

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

min_unemp = pa_counties['Unemp Rate'].min()
max_unemp = pa_counties['Unemp Rate'].max()
norm = Normalize(vmin=min_unemp, vmax=max_unemp * 1.2)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Unemp Rate',
    cmap='viridis',
    edgecolor='black',
    linewidth=0.5,
    legend=False,
    norm=norm,
    ax=ax
)

sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Unemployment Rate (%)", fontsize=10, labelpad=10)

cities_gdf.plot(ax=ax, color='white', markersize=50, label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)
legend_elements = [Line2D([0], [0], marker='o', color='black', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.title("Unemployment Rate by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
import seaborn as sns
import matplotlib.pyplot as plt

refined_vars = [
    'Median Household Income',
    'Per Capita Income',
    'Median Housing Value',
    'Population Below Poverty Line',
    'Percent_Bachelors',
    'Percent_Poverty',
    'Unemp Rate',
    'Percent Local Park'
]

refined_correlation_matrix = pa_counties[refined_vars].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    refined_correlation_matrix, 
    annot=True, 
    cmap='coolwarm', 
    fmt='.2f', 
    linewidths=0.5, 
    cbar_kws={"shrink": 0.8, "label": "Correlation Coefficient"}
)
plt.title("Correlation Matrix of Key Economic, Demographic, and Park Variables", fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
fig, ax = plt.subplots(figsize=(10, 6))

scatter = sns.scatterplot(
    data=pa_counties, 
    x='Percent_Bachelors', 
    y='Median Household Income', 
    size='Total Population', 
    hue='Unemp Rate',  
    palette='coolwarm', 
    sizes=(20, 200),  
    edgecolor='black',
    linewidth=0.5,
    alpha=0.8, 
    ax=ax
)

norm = plt.Normalize(pa_counties['Unemp Rate'].min(), pa_counties['Unemp Rate'].max())
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, aspect=50, shrink=0.75, pad=0.02)
cbar.set_label("Unemployment Rate (%)")

plt.title("Median Income vs. Bachelor's Degree Percent\nSize Reflects Population, Color Reflects Unemployment Rate")
plt.xlabel("Percentage of Adults with Bachelor's Degree")
plt.ylabel("Median Household Income")

sns.despine()

plt.tight_layout()
plt.show()

In [ ]:
#| echo: true
#| code-fold: true
pa_counties['Median Household Income'] = pd.to_numeric(pa_counties['Median Household Income'], errors='coerce')
pa_counties['Percent_Bachelors'] = pd.to_numeric(pa_counties['Percent_Bachelors'], errors='coerce')
pa_counties['Percent_Poverty'] = pd.to_numeric(pa_counties['Percent_Poverty'], errors='coerce')
pa_counties['Median Household Income'].fillna(0, inplace=True)
pa_counties['Percent_Bachelors'].fillna(0, inplace=True)
pa_counties['Percent_Poverty'].fillna(0, inplace=True)

scaler = MinMaxScaler()
variables = pa_counties[['Median Household Income', 'Percent_Bachelors', 'Percent_Poverty']].copy()
variables = pd.DataFrame(scaler.fit_transform(variables), columns=variables.columns)

pa_counties['Socioeconomic_Index'] = (
    variables['Median Household Income'] +
    variables['Percent_Bachelors'] -
    variables['Percent_Poverty']
)

pa_counties = pa_counties.to_crs(epsg=2272)
key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

norm = Normalize(vmin=pa_counties['Socioeconomic_Index'].min(), vmax=pa_counties['Socioeconomic_Index'].max())
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    column='Socioeconomic_Index',
    cmap='coolwarm',
    edgecolor='black',
    linewidth=0.5,
    norm=norm,
    legend=True,
    legend_kwds={'shrink': 0.6, 'label': "Socioeconomic Index"},
    ax=ax
)

cities_gdf.plot(ax=ax, color='black', markersize=50, label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")
ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)

legend_elements = [Line2D([0], [0], marker='o', color='red', markersize=8, linestyle='', label='Key Cities')]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)
plt.title("Socioeconomic Index by County in Pennsylvania\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold')
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
#| echo: false
# This chunk is hidden on the webiste. It just reloads the pa_counties dataframe in this notebook.
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import contextily as ctx
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

API_KEY = '0da0c882151e10740c1a0a844cf845096bedb565'
c = Census(API_KEY)

acs_data = c.acs5.state_county(
    fields=[
        'NAME',         # County name
        'B01003_001E',  # Total population
        'B15003_001E',  # Population 25+
        'B19013_001E',  # Median household income
        'B19301_001E',  # Per capita income
        'B01002_001E',  # Median age
        'B02001_002E',  # White population
        'B02001_003E',  # African American population
        'B03001_003E',  # Hispanic population
        'B15003_017E',  # High school graduates
        'B15003_022E',  # Bachelor’s degree holders
        'B25077_001E',  # Median housing value
        'B17001_002E',  # Population below poverty line
    ],
    state_fips=states.PA.fips,
    county_fips="*"
)

acs_df = pd.DataFrame(acs_data)
acs_df.columns = [
    'County',  # Name of the county
    'Total Population',
    'Population Over 25',
    'Median Household Income', 
    'Per Capita Income', 
    'Median Age', 
    'White Population', 
    'African American Population', 
    'Hispanic Population', 
    'High School Graduates', 
    'Bachelors Degree Holders', 
    'Median Housing Value', 
    'Population Below Poverty Line',
    'State',  
    'County_Name'
]

acs_df['County'] = acs_df['County'].str.replace(" County, Pennsylvania", "", case=False)
acs_df['County'] = acs_df['County'].str.upper()
acs_df.drop(columns=['State'], inplace=True)

geojson_path = "/Users/ryanswett/Downloads/Python/Final_Project/PaCounty2024_11.geojson"
county_data = gpd.read_file(geojson_path)
counties = county_data.merge(
    acs_df,
    left_on='FIPS_COUNT',
    right_on='County_Name',
    how='left'
)

counties['Percent_White'] = counties['White Population'] / counties['Total Population']*100 # Percent white
counties['Percent_Black'] = counties['African American Population'] / counties['Total Population']*100 # Percent black
counties['Percent_Hispanic'] = counties['Hispanic Population'] / counties['Total Population']*100 # Percent hispanic
counties['Percent_HS_degrees'] = counties['High School Graduates'] / counties['Total Population']*100 # Percent high school grads
counties['Percent_Bachelors'] = counties['Bachelors Degree Holders'] / counties['Population Over 25']*100 # Percent bachelors
counties['Percent_Poverty'] = counties['Population Below Poverty Line'] / counties['Population Over 25']*100 # Percent below poverty line

parks_path = "/Users/ryanswett/Downloads/Python/Final_Project/DCNR_LocalPark202406/DCNR_LocalPark202406.shp"
parks_data = gpd.read_file(parks_path)

parks_data = parks_data.drop(columns=['STATUS', 'PARK_FEE', 'ALT_NAME', 'PREMISE_AD', 'PREMISE_CI', 'PREMISE_ZI', 'YEAR_OPEN',
                                     'PREMISE_CR', 'URL', 'COMMENTS', 'ATV', 'Basketball', 'Bicycling', 'Camping', 'Canoeing_K',
                                     'CrossCount', 'Disc_Golf', 'Dog_Park', 'Equestrian', 'Fishing', 'Fitness_Eq', 'Golf',
                                     'Hiking', 'Horseback_', 'Hunting', 'Ice_Fishin', 'Ice_Skatin', 'Motor_Boat', 'LWCF_Restr',
                                     'Mountain_B', 'Natural_Wi', 'Organized_', 'Parking', 'Pavilion', 'Pets_Allow', 'Playground',
                                     'Restrooms', 'Rock_Climb', 'Scenic_Vie', 'Sledding', 'Sports_Fie', 'Swimming', 'Tennis_Cou',
                                     'Theatre_Am', 'Trails', 'Visitor_Ce', 'White_Wate', 'Wildlife_W', 'Amenity_Co', 'Feedback_l',
                                     'Skate_Park'])

updated_parks = parks_data.groupby('PREMISE_CO')['Acres'].sum().reset_index()
updated_parks['park_sq_mi'] = updated_parks['Acres'] / 640
updated_parks['PREMISE_CO'] = updated_parks['PREMISE_CO'].str.upper()

parks = counties.merge(
    updated_parks,
    left_on='County',  
    right_on='PREMISE_CO',  
    how='left')

parks['Percent Local Park'] = parks['park_sq_mi'] / parks['AREA_SQ_MI'] * 100

unemp_data = pd.read_csv("/Users/ryanswett/Downloads/Python/Final_Project/unemp.csv")
unemp_data['County'] = unemp_data['County'].str.replace('County', '', case=False).str.strip()
unemp_data['County'] = unemp_data['County'].str.upper()

pa_counties = unemp_data.merge(
    parks,
    left_on='County', 
    right_on='County', 
    how='left')

pa_counties.rename(columns={"Value (Percent)": "Unemp Rate"}, inplace=True)

pa_counties['Median Household Income'] = pd.to_numeric(pa_counties['Median Household Income'], errors='coerce')
pa_counties['Percent_Bachelors'] = pd.to_numeric(pa_counties['Percent_Bachelors'], errors='coerce')
pa_counties['Percent_Poverty'] = pd.to_numeric(pa_counties['Percent_Poverty'], errors='coerce')
pa_counties['Median Household Income'].fillna(0, inplace=True)
pa_counties['Percent_Bachelors'].fillna(0, inplace=True)
pa_counties['Percent_Poverty'].fillna(0, inplace=True)

scaler = MinMaxScaler()
variables = pa_counties[['Median Household Income', 'Percent_Bachelors', 'Percent_Poverty']].copy()
variables = pd.DataFrame(scaler.fit_transform(variables), columns=variables.columns)

pa_counties['Socioeconomic_Index'] = (
    variables['Median Household Income'] +
    variables['Percent_Bachelors'] -
    variables['Percent_Poverty']
)

pa_counties = gpd.GeoDataFrame(pa_counties, geometry=pa_counties['geometry'])

# Ensure the GeoDataFrame has a CRS
if pa_counties.crs is None:
    pa_counties = pa_counties.set_crs(epsg=4326)

# Reproject to Pennsylvania State Plane (EPSG: 2272)
pa_counties = pa_counties.to_crs(epsg=2272)

# Ensure all variables are numeric and handle NaN values
pa_counties['Median Household Income'] = pd.to_numeric(pa_counties['Median Household Income'], errors='coerce')
pa_counties['Percent_Bachelors'] = pd.to_numeric(pa_counties['Percent_Bachelors'], errors='coerce')
pa_counties['Percent_Poverty'] = pd.to_numeric(pa_counties['Percent_Poverty'], errors='coerce')

# Fill NaN values
pa_counties['Median Household Income'].fillna(0, inplace=True)
pa_counties['Percent_Bachelors'].fillna(0, inplace=True)
pa_counties['Percent_Poverty'].fillna(0, inplace=True)

# Normalize variables
scaler = MinMaxScaler()
variables = pa_counties[['Median Household Income', 'Percent_Bachelors', 'Percent_Poverty']].copy()
variables = pd.DataFrame(scaler.fit_transform(variables), columns=variables.columns)

# Calculate Socioeconomic Index
pa_counties['Socioeconomic_Index'] = (
    variables['Median Household Income'] +
    variables['Percent_Bachelors'] -
    variables['Percent_Poverty']
)

In [ ]:
#| echo: true
#| code-fold: true
clustering_vars = [
    'Socioeconomic_Index', 
    'Median Household Income', 
    'Percent_Bachelors', 
    'Percent_Poverty', 
    'Percent Local Park',
    'Unemp Rate'  
]

pa_counties[clustering_vars] = pa_counties[clustering_vars].apply(pd.to_numeric, errors='coerce')
for var in clustering_vars:
    pa_counties[var].fillna(pa_counties[var].median(), inplace=True)

scaler = StandardScaler()
pa_counties[clustering_vars] = scaler.fit_transform(pa_counties[clustering_vars])

pa_counties = pa_counties[pa_counties['geometry'].notnull()]

kmeans = KMeans(n_clusters=4, random_state=42)
pa_counties['Cluster'] = kmeans.fit_predict(pa_counties[clustering_vars])
pa_counties['Cluster'] = pa_counties['Cluster'] + 1  

key_cities = [
    {"name": "Philadelphia", "x": -75.1652, "y": 39.9526},
    {"name": "Harrisburg", "x": -76.8867, "y": 40.2732},
    {"name": "Pittsburgh", "x": -79.9959, "y": 40.4406}
]

cities_gdf = gpd.GeoDataFrame(
    key_cities,
    geometry=gpd.points_from_xy([city['x'] for city in key_cities], [city['y'] for city in key_cities]),
    crs="EPSG:4326"
).to_crs(epsg=2272)

pastel_colors = {
    1: "#1f77b4",  
    2: "#2ca02c",  
    3: "#d62728",  
    4: "#9467bd"  
}
pa_counties['color'] = pa_counties['Cluster'].map(pastel_colors)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
pa_counties.plot(
    color=pa_counties['color'],
    edgecolor='black',
    linewidth=0.5,
    ax=ax
)

cities_gdf.plot(ax=ax, color='white', edgecolor='black', markersize=50, label="Key Cities")
for city, geom in zip(key_cities, cities_gdf.geometry):
    ax.text(geom.x + 5000, geom.y, city["name"], fontsize=10, color='white', fontweight='bold', ha="left")

ctx.add_basemap(ax, crs=pa_counties.crs, source=ctx.providers.CartoDB.DarkMatter)

legend_elements = [
    Patch(facecolor=pastel_colors[1], edgecolor='black', label='Cluster 1'),
    Patch(facecolor=pastel_colors[2], edgecolor='black', label='Cluster 2'),
    Patch(facecolor=pastel_colors[3], edgecolor='black', label='Cluster 3'),
    Patch(facecolor=pastel_colors[4], edgecolor='black', label='Cluster 4'),
    Patch(facecolor='white', edgecolor='black', label='Key Cities')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10, labelcolor='white')


plt.title("County Clusters Based on Socioeconomic, Demographic, Park, and Unemployment Variables\nKey Cities Highlighted", 
          fontsize=14, fontweight='bold', color='white')
ax.axis("off")
plt.tight_layout()
plt.show()

cluster_summary = pa_counties.groupby('Cluster')[clustering_vars].mean()
print("Cluster Summary:")
print(cluster_summary)

In [ ]:
#| echo: true
#| code-fold: true
cluster_summary = pa_counties.groupby('Cluster')[[
    'Median Household Income', 
    'Population Below Poverty Line', 
    'Percent_Bachelors', 
    'Percent Local Park',
    'Unemp Rate',
    'Total Population'
]].mean()

cluster_summary = cluster_summary.rename(columns={
    'Median Household Income': 'Median Household Income ($)',
    'Population Below Poverty Line': 'Population Below Poverty Line (%)',
    'Percent_Bachelors': "Bachelor's Degree Holders (%)",
    'Percent Local Park': 'Local Park Coverage (%)',
    'Unemp Rate': 'Unemployment Rate (%)',
    'Total Population': 'Total Population'
})

print("Within-Cluster Characteristics Summary:")
print(cluster_summary)

sns.set_theme(style="darkgrid", rc={"axes.facecolor": "#222222", "figure.facecolor": "#222222"})

cluster_colors = {
    1: "#1f77b4",  
    2: "#2ca02c",  
    3: "#d62728",  
    4: "#9467bd"   
}
colors = [cluster_colors[i] for i in range(1, 5)]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle("Cluster Characteristics by Demographic, Economic, and Environmental Indicators", fontsize=16, fontweight='bold', color='white')

variables = cluster_summary.columns
for ax, var in zip(axes.flatten(), variables):
    bars = cluster_summary[var].plot(kind='bar', ax=ax, color=colors, width=0.8)
    ax.set_title(var, fontsize=12, fontweight='bold', color='white')
    ax.set_xlabel("Cluster", fontsize=10, color='white')
    ax.set_ylabel("Average Value (z-score)", fontsize=10, color='white')
    ax.tick_params(axis='x', rotation=0, colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.grid(axis='y', linestyle='--', alpha=0.5, color='gray')  
    
    for bar in bars.patches:
        ax.text(
            bar.get_x() + bar.get_width() / 2, 
            bar.get_height() + 0.02 * max(cluster_summary[var]), 
            f"{bar.get_height():.2f}", 
            ha='center', fontsize=8, color='white'
        )

for ax in axes.flatten()[len(variables):]:
    ax.axis("off")
    ax.set_facecolor('#222222')
    
plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.show()